원본 출처: https://github.com/rickiepark/deep-learning-with-python-notebooks <a href="https://github.com/rickiepark/deep-learning-with-python-notebooks/blob/master/LICENSE">(MIT License)</a>

In [ ]:
import keras
keras.__version__

'2.8.0'

# 신경망과의 두 번째 만남

이 노트북은 [케라스 창시자에게 배우는 딥러닝](https://tensorflow.blog/케라스-창시자에게-배우는-딥러닝/) 책의 2장 1절의 코드 예제입니다. 책에는 더 많은 내용과 그림이 있습니다. 이 노트북에는 소스 코드에 관련된 설명만 포함합니다. 이 노트북의 설명은 케라스 버전 2.2.2에 맞추어져 있습니다. 케라스 최신 버전이 릴리스되면 노트북을 다시 테스트하기 때문에 설명과 코드의 결과가 조금 다를 수 있습니다.

----

케라스 파이썬 라이브러리를 사용하여 손글씨 숫자 분류를 학습하는 구체적인 신경망 예제를 살펴보겠습니다. 케라스나 비슷한 라이브러리를 사용한 경험이 없다면 당장은 이 첫 번째 예제를 모두 이해하지 못할 것입니다. 아직 케라스를 설치하지 않았을지도 모릅니다. 괜찮습니다. 다음 장에서 이 예제를 하나하나 자세히 설명합니다. 코드가 좀 이상하거나 요술처럼 보이더라도 너무 걱정하지 마세요. 일단 시작해 보겠습니다.

여기에서 풀려고 하는 문제는 흑백 손글씨 숫자 이미지(28x28 픽셀)를 10개의 범주(0에서 9까지)로 분류하는 것입니다. 머신 러닝 커뮤니티에서 고전으로 취급받는 데이터셋인 MNIST를 사용하겠습니다. 이 데이터셋은 머신 러닝의 역사만큼 오래되었고 많은 연구에 사용되었습니다. 이 데이터셋은 1980년대에 미국 국립표준기술연구소에서 수집한 6만 개의 훈련 이미지와 1만 개의 테스트 이미지로 구성되어 있습니다. MNIST 문제를 알고리즘이 제대로 작동하는지 확인하기 위한 딥러닝계의 ‘Hello World’라고 생각해도 됩니다. 머신 러닝 기술자가 되기까지 연구 논문이나 블로그 포스트 등에서 MNIST를 보고 또 보게 될 것입니다.

MNIST 데이터셋은 넘파이 배열 형태로 케라스에 이미 포함되어 있습니다:

In [1]:
# Keras에서 제공하는 MNIST 데이터셋 불러오기
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


`train_images`와 `train_labels`가 모델이 학습해야 할 훈련 세트를 구성합니다. 모델은 `test_images`와 `test_labels`로 구성된 테스트 세트에서 테스트될 것입니다. 이미지는 넘파이 배열로 인코딩되어 있고 레이블은 0에서부터 9까지의 숫자 배열입니다. 이미지와 레이블은 일대일 관계를 가집니다.



작업 순서는 다음과 같습니다. 먼저 훈련 데이터 `train_images`와 `train_labels`를 네트워크에 주입합니다. 그러면 네트워크는 이미지와 레이블을 연관시킬 수 있도록 학습됩니다. 마지막으로 `test_images`에 대한 예측을 네트워크에게 요청합니다. 그리고 이 예측이 `test_labels`와 맞는지 확인할 것입니다.

신경망을 만들어 보겠습니다. 

In [2]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,))) 
network.add(layers.Dense(10, activation='softmax'))

In [ ]:
# 네트워크의 구조 보기
# 첫 번째 층 가중치 수 419120 = 입력 노드 (785 = 28*28 + 1) * 히든 노드 (512)
# 두 번째 층 가중치 수   5130 = 히든 노드 (512 + 1) * 출력 노드 (10)

network.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


신경망의 핵심 구성 요소는 일종의 데이터 처리 필터라고 생각할 수 있는 층입니다. 어떤 데이터가 들어가면 더 유용한 형태로 출력됩니다. 조금 더 구체적으로 층은 주어진 문제에 더 의미 있는 표현을 입력된 데이터로부터 추출합니다. 대부분의 딥러닝은 간단한 층을 연결하여 구성되어 있고, 점진적으로 데이터를 정제하는 형태를 띠고 있습니다. 딥러닝 모델은 데이터 정제 필터(층)가 연속되어 있는 데이터 프로세싱을 위한 여과기와 같습니다.

이 예에서는 조밀하게 연결된 (또는 완전 연결된) 신경망 층인 `Dense` 층 2개가 연속되어 있습니다. 두 번째 (즉, 마지막) 층은 10개의 확률 점수가 들어 있는 배열(모두 더하면 1입니다)을 반환하는 소프트맥스 층입니다. 각 점수는 현재 숫자 이미지가 10개의 숫자 클래스 중 하나에 속할 확률입니다.

신경망이 훈련 준비를 마치기 위해서 컴파일 단계에 포함될 세 가지가 더 필요합니다:

* 손실 함수 : 훈련 데이터에서 신경망의 성능을 측정하는 방법으로 네트워크가 옳은 방향으로 학습될 수 있도록 도와 줍니다.
* 옵티마이저: 입력된 데이터와 손실 함수를 기반으로 네트워크를 업데이트하는 메커니즘입니다.
* 훈련과 테스트 과정을 모니터링할 지표 : 여기에서는 정확도(정확히 분류된 이미지의 비율)만 고려하겠습니다.

손실 함수와 옵티마이저의 정확한 목적은 이어지는 두 개의 장에서 자세히 설명하겠습니다.

In [3]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

훈련을 시작하기 전에 데이터를 네트워크에 맞는 크기로 바꾸고 모든 값을 0과 1 사이로 스케일을 조정합니다. 예를 들어, 앞서 우리의 훈련 이미지는 `[0, 255]` 사이의 값인 `uint8` 타입의 `(60000, 28, 28)` 크기를 가진 배열로 저장되어 있습니다. 이 데이터를 0과 1 사이의 값을 가지는 `float32` 타입의 `(60000, 28 * 28)` 크기의 배열로 바꿉니다.

In [4]:
norm_train_images = train_images.reshape((60000, 28 * 28))
norm_train_images = norm_train_images.astype('float32') / 255

norm_test_images = test_images.reshape((10000, 28 * 28))
norm_test_images = norm_test_images.astype('float32') / 255

또한, 레이블을 범주형으로 인코딩해야 합니다. 이 단계는 3장에서 자세히 설명하겠습니다:

In [5]:
from tensorflow.keras.utils import to_categorical

cat_train_labels = to_categorical(train_labels)
cat_test_labels = to_categorical(test_labels)

이제 신경망을 훈련시킬 준비가 되었습니다. 케라스에서는 `fit` 메서드를 호출하여 훈련 데이터에 모델을 학습시킵니다:

In [6]:
network.fit(norm_train_images, cat_train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 6s 11ms/step - loss: 0.2579 - accuracy: 0.9261
Epoch 2/5
469/469 [==============================] - 5s 11ms/step - loss: 0.1039 - accuracy: 0.9690
Epoch 3/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0676 - accuracy: 0.9801
Epoch 4/5
469/469 [==============================] - 5s 11ms/step - loss: 0.0489 - accuracy: 0.9857
Epoch 5/5
469/469 [==============================] - 5s 11ms/step - loss: 0.0372 - accuracy: 0.9884


훈련하는 동안 두 개의 정보가 출력됩니다. 훈련 데이터에 대한 네트워크의 손실과 정확도입니다.

훈련 데이터에 대해 0.989(98.9%)의 정확도를 금방 달성합니다. 이제 테스트 세트에서도 모델이 잘 작동하는지 확인해 보겠습니다:

In [ ]:
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.0708 - accuracy: 0.9804
INFO:tensorflow:Assets written to: mobilenet_imagenet_model/assets


1629664

In [ ]:
print('test_acc:', test_acc)

test_acc: 0.9746999740600586


테스트 세트의 정확도는 97.8%로 나왔습니다. 훈련 세트 정확도보다는 약간 낮습니다. 훈련 정확도와 테스트 정확도 사이의 차이는 과대적합 때문입니다. 이는 머신 러닝 모델이 훈련 데이터보다 새로운 데이터에서 성능이 낮아지는 경향을 말합니다. 과대적합은 3장에서 자세하게 논의하겠습니다.

이것으로 첫 번째 예제가 마무리되었습니다. 20줄 미만의 파이썬 코드로 손글씨 숫자를 분류하는 신경망을 만들고 훈련시켰습니다. 다음 장에서 여기서 보았던 코드 하나하나를 상세하게 설명하고 이들이 의미하는 바를 명확하게 설명하겠습니다. 이제 텐서, 신경망에 주입하는 데이터의 저장 형태, 층을 만들어주는 텐서 연산, 신경망을 훈련 샘플로부터 학습시키는 경사 하강법에 대해 알아보겠습니다.

## TODO 1: 은닉층의 유닛수
1. 첫 번째 은닉층의 유닛수를 64로 바꾸면 테스트 데이터 정확도는?
2. 첫 번째 은닉층의 유닛수를 1024로 바꾸면 테스트 데이터 정확도는?

In [ ]:
# 64일 때
from tensorflow.keras import optimizers
network = models.Sequential()

# 층, 함수, input
network.add(layers.Dense(64, activation='relu',input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(norm_train_images, cat_train_labels, epochs=10, batch_size=128)
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)
print('test_acc:', test_acc)

NameError: ignored

In [ ]:
# 1024일 때
from tensorflow.keras import optimizers
network = models.Sequential()
network.add(layers.Dense(1024, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(norm_train_images, cat_train_labels, epochs=10, batch_size=128)
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)
print('test_acc:', test_acc)

Epoch 1/10
469/469 [==============================] - 8s 16ms/step - loss: 0.2349 - accuracy: 0.9307
Epoch 2/10
469/469 [==============================] - 7s 16ms/step - loss: 0.0880 - accuracy: 0.9736
Epoch 3/10
469/469 [==============================] - 7s 16ms/step - loss: 0.0577 - accuracy: 0.9825
Epoch 4/10
469/469 [==============================] - 7s 16ms/step - loss: 0.0402 - accuracy: 0.9878
Epoch 5/10
469/469 [==============================] - 7s 16ms/step - loss: 0.0289 - accuracy: 0.9912
Epoch 6/10
469/469 [==============================] - 7s 16ms/step - loss: 0.0221 - accuracy: 0.9934
Epoch 7/10
469/469 [==============================] - 7s 16ms/step - loss: 0.0158 - accuracy: 0.9952
Epoch 8/10
469/469 [==============================] - 7s 16ms/step - loss: 0.0120 - accuracy: 0.9962
Epoch 9/10
469/469 [==============================] - 7s 16ms/step - loss: 0.0091 - accuracy: 0.9974
Epoch 10/10
313/313 [==============================] - 1s 3ms/step - loss: 0.0762 - accurac

## TODO 2: 은닉층의 활성화 함수
1. 첫 번째 은닉층의 활성화 함수를 sigmoid로 바꾸면 테스트 데이터 정확도는?
2. 첫 번째 은닉층의 활성화 함수를 tanh로 바꾸면 테스트 데이터 정확도는?

In [ ]:
# sigmoid일 때
from tensorflow.keras import optimizers
network = models.Sequential()
network.add(layers.Dense(512, activation='sigmoid', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(norm_train_images, cat_train_labels, epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)
print('test_acc:', test_acc)

Epoch 1/5
469/469 [==============================] - 5s 11ms/step - loss: 0.4255 - accuracy: 0.8848
Epoch 2/5
469/469 [==============================] - 5s 10ms/step - loss: 0.2357 - accuracy: 0.9317
Epoch 3/5
469/469 [==============================] - 4s 9ms/step - loss: 0.1800 - accuracy: 0.9469
Epoch 4/5
469/469 [==============================] - 5s 10ms/step - loss: 0.1416 - accuracy: 0.9586
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.1168 - accuracy: 0.9652
test_acc: 0.9652000069618225


In [ ]:
# tanh일 때
from tensorflow.keras import optimizers
network = models.Sequential()
network.add(layers.Dense(512, activation='tanh', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(norm_train_images, cat_train_labels, epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)
print('test_acc:', test_acc)

Epoch 1/5
469/469 [==============================] - 5s 9ms/step - loss: 0.3371 - accuracy: 0.9006
Epoch 2/5
469/469 [==============================] - 4s 9ms/step - loss: 0.1730 - accuracy: 0.9489
Epoch 3/5
469/469 [==============================] - 4s 9ms/step - loss: 0.1167 - accuracy: 0.9654
Epoch 4/5
469/469 [==============================] - 4s 9ms/step - loss: 0.0853 - accuracy: 0.9748
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0764 - accuracy: 0.9763
test_acc: 0.9763000011444092


## TODO 3: Softmax
1. [-1, 0, 1]에서 softmax 결과 0이 선택될 확률은?
2. [1, 2, 3]에서 softmax 결과 3이 선택될 확률은?
3. [10, 20, 30]에서 softmax 결과 30이 선택될 확률은?

In [ ]:
from scipy.special import softmax

# 1번
# print(softmax([-1,0,1]))
print(0.09003057)

# 2번
# print(softmax([1,2,3]))
print(0.66524096)

# 3번
# print(softmax([10,20,30]))
print(9.99954600e-01)

[0.09003057 0.24472847 0.66524096]
[0.09003057 0.24472847 0.66524096]
[2.06106005e-09 4.53978686e-05 9.99954600e-01]


## TODO 4: Optimizer
https://keras.io/api/optimizers/sgd/
1. optimizer에 SGD()를 썼을 때 테스트 정확도는?
2. optimizer에 SGD(learning_rate=0.1, momentum=0.9, nesterov=True)를 썼을 때 테스트 정확도는?
3. optimizer에 Adam()을 썼을 때 테스트 정확도는?

In [ ]:
# SGD()
from tensorflow.keras import optimizers
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='SGD',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(norm_train_images, cat_train_labels, epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)
print('test_acc:', test_acc)

Epoch 1/5
469/469 [==============================] - 4s 8ms/step - loss: 1.1013 - accuracy: 0.7617
Epoch 2/5
469/469 [==============================] - 4s 8ms/step - loss: 0.5226 - accuracy: 0.8741
Epoch 3/5
469/469 [==============================] - 3s 7ms/step - loss: 0.4203 - accuracy: 0.8911
Epoch 4/5
469/469 [==============================] - 3s 7ms/step - loss: 0.3740 - accuracy: 0.8991
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.3188 - accuracy: 0.9147
test_acc: 0.9146999716758728


In [ ]:
# SGD(learning_rate=0.1, momentum=0.9, nesterov=True)
from tensorflow.keras import optimizers
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

sgd = optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=True)

network.compile(optimizer='sgd',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(norm_train_images, cat_train_labels, epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)
print('test_acc:', test_acc)

Epoch 1/5
469/469 [==============================] - 4s 7ms/step - loss: 1.0840 - accuracy: 0.7611
Epoch 2/5
469/469 [==============================] - 3s 7ms/step - loss: 0.5191 - accuracy: 0.8722
Epoch 3/5
469/469 [==============================] - 3s 7ms/step - loss: 0.4187 - accuracy: 0.8900
Epoch 4/5
469/469 [==============================] - 3s 7ms/step - loss: 0.3727 - accuracy: 0.8994
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.3151 - accuracy: 0.9167
test_acc: 0.916700005531311


In [ ]:
# Adam()
from tensorflow.keras import optimizers
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='Adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(norm_train_images, cat_train_labels, epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)
print('test_acc:', test_acc)

Epoch 1/5
469/469 [==============================] - 4s 9ms/step - loss: 0.2691 - accuracy: 0.9232
Epoch 2/5
469/469 [==============================] - 3s 7ms/step - loss: 0.1078 - accuracy: 0.9680
Epoch 3/5
469/469 [==============================] - 3s 7ms/step - loss: 0.0688 - accuracy: 0.9798
Epoch 4/5
469/469 [==============================] - 3s 7ms/step - loss: 0.0501 - accuracy: 0.9852
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.0718 - accuracy: 0.9766
test_acc: 0.9765999913215637


## TODO 5: 정규화의 영향

1. 이미지들을 정규화하지 않고 학습시켰을 때 테스트 정확도는?

In [ ]:
# 이미지 reshape
conv_train_images = train_images.reshape((60000, 28 * 28))
conv_test_images = test_images.reshape((10000, 28 * 28))

from tensorflow.keras import optimizers
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(conv_train_images, cat_train_labels, epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(conv_test_images, cat_test_labels)
print('test_acc:', test_acc)

Epoch 1/5
469/469 [==============================] - 5s 10ms/step - loss: 5.9698 - accuracy: 0.8976
Epoch 2/5
469/469 [==============================] - 4s 9ms/step - loss: 0.7191 - accuracy: 0.9542
Epoch 3/5
469/469 [==============================] - 5s 10ms/step - loss: 0.4893 - accuracy: 0.9649
Epoch 4/5
469/469 [==============================] - 4s 9ms/step - loss: 0.3765 - accuracy: 0.9719
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.7385 - accuracy: 0.9626
test_acc: 0.9625999927520752


## TODO 6: 최적화

1. MLP로 Test 정확도를 98.2% 이상으로 만들어보세요. (층 추가 가능)

In [ ]:
from tensorflow.keras import optimizers
network = models.Sequential()
network.add(layers.Dense(2000, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(norm_train_images, cat_train_labels, epochs=10, batch_size=128)
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)
print('test_acc:', test_acc)

Epoch 1/10
469/469 [==============================] - 14s 30ms/step - loss: 0.2180 - accuracy: 0.9341
Epoch 2/10
469/469 [==============================] - 14s 30ms/step - loss: 0.0802 - accuracy: 0.9756
Epoch 3/10
469/469 [==============================] - 14s 30ms/step - loss: 0.0516 - accuracy: 0.9840
Epoch 4/10
469/469 [==============================] - 14s 30ms/step - loss: 0.0365 - accuracy: 0.9887
Epoch 5/10
469/469 [==============================] - 14s 30ms/step - loss: 0.0253 - accuracy: 0.9921
Epoch 6/10
469/469 [==============================] - 14s 30ms/step - loss: 0.0199 - accuracy: 0.9938
Epoch 7/10
469/469 [==============================] - 14s 30ms/step - loss: 0.0150 - accuracy: 0.9955
Epoch 8/10
469/469 [==============================] - 14s 30ms/step - loss: 0.0115 - accuracy: 0.9962
Epoch 9/10
469/469 [==============================] - 14s 30ms/step - loss: 0.0087 - accuracy: 0.9973
Epoch 10/10
313/313 [==============================] - 2s 5ms/step - loss: 0.0905 

##TODO 7: 마무리

오늘 실습에서 인공신경망의 성능에 영향을 준 요인들을 나열해보세요.

In [ ]:
# A:
# Dense - 몇 개의 유닛을 셋팅 / 어떤 함수를 사용
# compile - Optimizers(알고리즘) / Loss function
# fit - epochs(수행할 횟수)
# 정규화 - 정확성을 늘릴 수 있음